In [ ]:
import pandas as pd
import psycopg2
import json
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json

## Creando un mayor valor a nuestros datos
### Transformaciones

##### Identificamos los parámetros de análisis que tienen mayor influencia en la contaminación del agua:

In [ ]:
parametros_influencia = water.groupby('NombreParametroAnalisis2')['IrcaPromedio'].mean().sort_values(ascending=False)
top_15_parametros = parametros_influencia.head(15)
top_15_parametros


Todos estos parámetros tienen una media de IRCA de aproximadamente 23.94, lo que sugiere una asociación con un nivel de riesgo considerable. Esto puede ser útil para priorizar qué parámetros de calidad del agua necesitan atención más crítica en los programas de monitoreo y tratamiento.

In [ ]:
water = water[water['NombreParametroAnalisis2'].isin(top_15_parametros.index)]

water.head(), water.shape

Hemos filtrado el dataset para dejar solo las filas correspondientes al top 15 parámetros de análisis relacionados con la contaminación del agua y los demas han salido eliminados

##### Clasificación del Índice de Riesgo de la Calidad del Agua (IRCA)

In [ ]:
def clasificar_irca(irca):
    try:
        irca = float(irca.replace(',', '.'))
        if irca == 0:
            return 'Sin información'
        elif 0.001 <= irca <= 5:
            return 'Sin riesgo'
        elif 5.001 <= irca <= 14:
            return 'Riesgo bajo'
        elif 14.001 <= irca <= 35:
            return 'Riesgo medio'
        elif 35.001 <= irca <= 80:
            return 'Riesgo alto'
        elif 80.001 <= irca <= 100:
            return 'Inviable sanitariamente'
        else:
            return 'No clasificado'
    except ValueError:
        return 'No clasificado'

water['rango_irca'] = water['IrcaPromedio'].apply(clasificar_irca)

La función clasificar_irca transforma el valor numérico del IrcaPromedio en categorías descriptivas que van desde 'Sin riesgo' hasta 'Inviable sanitariamente', facilitando la interpretación y toma de decisiones en la gestión de la calidad del agua. Esta categorización es crucial para los análisis de salud pública y ambientales, ya que simplifica la visualización de los datos, permite una comparación rápida entre regiones y es esencial para modelar y prever la calidad del agua, lo que resulta en intervenciones más efectivas y políticas basadas en evidencia.

titulo de siguiente

In [ ]:
water = water.drop(['ResultadoMinimo', 'ResultadoMaximo', 'ResultadoPromedio'], axis=1)

In [ ]:
columnas_a_eliminar = ['MuestrasTratadas', 'MuestrasEvaluadas', 'MuestrasSinTratar',
                      'NumeroParametrosMinimo', 'NumeroParametrosMaximo', 'EsAtipico']
water = water.drop(columns=columnas_a_eliminar)


water.head()

In [ ]:
new_column_names = [
    "año", 
    "nombre_departamento", 
    "div_dpto", 
    "nombre_municipio", 
    "divi_muni", 
    "irca_minimo", 
    "irca_maximo", 
    "irca_promedio", 
    "rango_irca", 
    "nombre_parametro_analisis", 
    "numero_parametros_promedio", 
    "rango_parametros_analizados",
    "diferencia_muestras_tratadas_sin_tratar"
    
]
water.columns = new_column_names
print(water.head())